In [62]:
import torch
from torch import nn, optim
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
        
    def num_flat_features(self, x):
        size = x.size()[1:]
        
        num_features = 1
        for s in size:
            num_features *= s
        
        return num_features

In [52]:
n = Net()

In [53]:
n

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [54]:
x_input = torch.randn(1, 1, 32, 32)
out = n(x_input)

In [55]:
out

tensor([[-0.0323,  0.0152,  0.0865,  0.0575,  0.0634, -0.0410,  0.0727, -0.0496,
          0.0283, -0.0554]], grad_fn=<AddmmBackward>)

In [45]:
n.zero_grad()
out.backward(torch.randn(1, 10))

In [56]:
target = torch.randn(1, 10)

In [57]:
criterion = nn.MSELoss()
loss = criterion(out, target)

In [58]:
loss

tensor(0.3197, grad_fn=<MseLossBackward>)

In [59]:
loss.grad_fn

In [60]:
n.zero_grad()
loss.backward()

In [61]:
n.conv1.bias.grad

tensor([-0.0058,  0.0032,  0.0044,  0.0017, -0.0087,  0.0035])

In [63]:
optimizer = optim.SGD(n.parameters(), lr=0.01)

In [64]:
optimizer.zero_grad()
out = n(x_input)
loss = criterion(out, target)
loss.backward()
optimizer.step()